In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re
import pyLDAvis.gensim_models
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
import json
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

C:\Users\User\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\User\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\User\anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\User\anaconda3\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


### LDA 참고: https://wikidocs.net/30708

In [2]:
df= pd.read_csv('maple_title_완.csv',encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817632 entries, 0 to 817631
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   817632 non-null  object
 1   date    817632 non-null  object
dtypes: object(2)
memory usage: 12.5+ MB


In [3]:
#광고, 설문 데이터 제거
values_to_remove = list(df['title'][0:3])
values_to_remove.append('클럽 가면 내 집처럼 잘 놀 것 같은 스타는?')
df = df[~df['title'].isin(values_to_remove)]
# index 초기화
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771201 entries, 0 to 771200
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   771201 non-null  object
 1   date    771201 non-null  object
dtypes: object(2)
memory usage: 11.8+ MB


In [4]:
def remove_non(text):
    # 정규 표현식을 사용하여 한글, 숫자, 알파벳과 띄어쓰기만 남김
    pattern = r'[^\w가-힣\s]+'  # 한글, 숫자, 알파벳, 띄어쓰기를 제외한 모든 문자 제거
    return re.sub(pattern, '', text)

In [5]:
df['clean_doc'] = df['title'].apply(remove_non)
# 길이가 1이하인 단어는 제거 (길이가 짧은 단어 제거)
df['clean_doc'] = df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
# # 전체 단어에 대한 소문자 변환
df['clean_doc'] = df['clean_doc'].apply(lambda x: x.lower())
#숫자제거
df['clean_doc'] = df['clean_doc'].apply(lambda x: re.sub(r'\d', '', x))

In [6]:
#불용어 출처:https://mr-doosun.tistory.com/24
stop_words = ['app','던파','이것은','순서','메이플','이유','있음','성','퍼','ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ','다들','뭐냐','뭐임','ㅅㅂ','이제','ㅈㄴ','개','억','차','요즘','너무','님들','아님','같은','가면','오늘','게임','ㄹㅇ','ㅋㅋㅋ','ㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋ','지금','근데','존나','아니','씨발','ㅋㅋ','그냥','카드맆','ㅇㅇ','이미지','dc','official','on','on2','on3','on4','on5','on6','아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리','있었어요','있었다','있습니다' '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를', '들면', '예를', '들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수', '없다', '해서는', '안된다', '뿐만', '아니라', '만이', '아니다', '만은', '아니다', '막론하고', '관계없이', '그치지', '않다', '그러나', '그런데', '하지만', '든간에', '논하지', '않다', '따지지', '않다', '설사', '비록', '더라도', '아니면', '만', '못하다', '하는', '편이', '낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이', '외에', '이', '밖에', '하여야', '비로소', '한다면', '몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할', '생각이다', '하려고하다', '이리하여', '그리하여', '그렇게', '함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는', '데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에', '틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해', '말하자면', '뿐이다', '다음에', '반대로', '반대로', '말하자면', '이와', '반대로', '바꾸어서', '말하면', '바꾸어서', '한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에', '가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한', '까닭에', '하기', '때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느곳', '더군다나', '하물며', '더욱이는', '어느때', '언제', '야', '이봐', '어이', '여보시오', '흐흐', '흥', '휴', '헉헉', '헐떡헐떡', '영차', '여차', '어기여차', '끙끙', '아야', '앗', '아야', '콸콸', '졸졸', '좍좍', '뚝뚝', '주룩주룩', '솨', '우르르', '그래도', '또', '그리고', '바꾸어말하면', '바꾸어서', '말하면', '바꾸어서', '한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에', '가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한', '까닭에', '하기', '때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느', '로써', '까지', '예하면', '했어요', '해요', '함께', '같이', '더불어', '마저', '마저도', '양자', '모두', '습니다', '가까스로', '하려고하다', '즈음하여', '다른', '다른', '방면으로', '해봐요', '습니까', '했어요', '말할것도', '없고', '무릎쓰고', '개의치않고', '하는것만', '못하다', '하는것이', '낫다', '매', '매번', '들', '모', '어느것', '어느', '로써', '갖고말하자면', '어디', '어느쪽', '어느것', '어느해', '어느', '년도', '라', '해도', '언젠가', '어떤것', '어느것', '저기', '저쪽', '저것', '그때', '그럼', '그러면', '요만한걸', '그래', '그때', '저것만큼', '그저', '이르기까지', '할', '지', '않는다면', '만약', '만일', '위에서', '서술한바와같이', '인', '듯하다', '하', '하지', '않는다면', '만약에', '무엇', '무슨', '어느', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '놓', '이거','진짜','시발']
#토큰화
tokenized_doc = df['clean_doc'].apply(lambda x: x.split())
#불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [7]:
#불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 실습 참고)는
#기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다.
#그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서
#다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다.
detokenized_doc = []
for i in range(len(df)):
    
    t = ' '.join(tokenized_doc[i])
    
    detokenized_doc.append(t)
    

df['clean_doc'] = detokenized_doc
df['clean_doc']

0                              그래서그냥메이플템싹정리한거지
1                                섀도어 분쇄닼사 병신같네
2                                    춘자 왜케 비호지
3                                           쓔발
4                                      턴테이블 만원
                          ...                 
771196    뉴비들한테 제일 진입장벽은 드메템인데 대여시스템이라도 만들어야할듯
771197                               물개새끼 안받앗다
771198                메린이인데 엠블렘 윗잠 주스탯 퍼뜨면 쓸만함
771199                            그래서패치 보통몇시에함
771200                             만원 남앗다 진짜뜨나
Name: clean_doc, Length: 771201, dtype: object

In [8]:
vectorizer = TfidfVectorizer(stop_words='english',max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (771201, 705486)


In [9]:
#토픽 개수
topic_num =10

In [10]:
# n_components : n개의 토픽을 가졌다고 가정
svd_model = TruncatedSVD(n_components=topic_num, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

10

In [11]:
np.shape(svd_model.components_)

(10, 705486)

In [12]:
terms = vectorizer.get_feature_names_out() # 단어 집합.

def get_topics(components, feature_names, n=topic_num):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('매크로', 0.70925), ('자동사냥', 0.70451), ('사냥', 0.01248), ('mshxxmxx', 0.01213), ('텔레그램', 0.01204), ('판매', 0.01115), ('채집', 0.00264), ('아르테일', 0.00212), ('큐브', 0.00187), ('농장', 0.00154)]
Topic 2: [('리부트', 0.97894), ('너프', 0.09141), ('본섭', 0.06609), ('최종뎀', 0.06505), ('ㄴ라고', 0.05653), ('vs', 0.04218), ('닉네임', 0.03974), ('생제', 0.03715), ('메린이', 0.03314), ('뉴비', 0.03097)]
Topic 3: [('닉네임', 0.91743), ('메린이', 0.20957), ('뉴비', 0.14546), ('질문', 0.12482), ('어떰', 0.12223), ('많관부', 0.08157), ('ㅁㅌㅊ', 0.07636), ('만원', 0.06508), ('팝니다', 0.06503), ('팔리냐', 0.05255)]
Topic 4: [('메린이', 0.71716), ('질문', 0.40212), ('뉴비', 0.38532), ('질문좀', 0.16515), ('유니온', 0.07765), ('vs', 0.07546), ('ㅁㅌㅊ', 0.04749), ('버닝섭', 0.0466), ('무기', 0.04112), ('어떰', 0.04089)]
Topic 5: [('뉴비', 0.73497), ('어떰', 0.39405), ('vs', 0.15805), ('코디', 0.04416), ('유니온', 0.03972), ('버닝섭', 0.03528), ('ㅁㅌㅊ', 0.03378), ('추천좀', 0.03305), ('사냥', 0.02786), ('뭐가', 0.02301)]
Topic 6: [('어떰', 0.89655), ('메린이', 0.1561), ('코디', 0.07668), ('사냥', 

In [13]:
documents = [doc.split() for doc in df['clean_doc']]
dictionary = corpora.Dictionary(documents)


corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

## 2) LDA 모델 훈련시키기

In [ ]:
import gensim
NUM_TOPICS = 5 # 5개의 토픽, k=5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)